### Setup

- Generate cooccurrences (value represents number of articles in which two companies occur together)

TODO:
- Other measures for cooccurrence?

In [1]:
import os
import re
import glob
from datetime import datetime
import sys
sys.path.append("..") # Adds higher directory to python modules path for importing from src dir

import pandas as pd
import numpy as np
import tqdm
import matplotlib
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook as tqdm

import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

from src.datasets import NyseStocksDataset, NyseSecuritiesDataset, NyseFundamentalsDataset
import src.econometric_utils as eco
import src.regression_utils as regr
import src.plot_utils as plot
import src.math_utils as math_utils
import src.nlp_utils as nlp_utils
import src.utils as utils

%matplotlib inline
%load_ext autotime
%load_ext autoreload
%autoreload 2

c:\users\thomas\hpi\ma\venv\lib\importlib\_bootstrap.py:205: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
c:\users\thomas\hpi\ma\venv\lib\importlib\_bootstrap.py:205: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)
c:\users\thomas\hpi\ma\venv\lib\importlib\_bootstrap.py:205: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
c:\users\thomas\hpi\ma\venv\lib\importlib\_bootstrap.py:205: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)


In [2]:
stocks_ds = NyseStocksDataset(file_path='../data/nyse/prices-split-adjusted.csv')
securities_ds = NyseSecuritiesDataset(file_path='../data/nyse/securities.csv')
companies = securities_ds.get_all_company_names()  # List[Tuple[symbol, name]]
stocks_ds.load()


time: 11 s


### Preprocess & Validate

In [ ]:
NEWS = os.path.join("..", "data", "preprocessed", "news-v3.csv")
articles = pd.read_csv(NEWS, index_col=0)
# articles = pd.read_csv(NEWS, index_col=0, nrows=people_occ.article_id.max()+50)
# articles = pd.read_csv(NEWS, index_col=0, skiprows=range(1, people_occ.article_id.max()))
# articles = articles[articles.index < 106519]

In [ ]:
def find_increases(people_occ, article, add=0, strict=False):
    increases = {}
    pbar = tqdm(articles.index)
    for article_id in pbar:
        for i, row in people_occ[people_occ.article_id == article_id].iterrows():
            if pd.isna(articles.loc[article_id+add].content) or articles.loc[article_id+add].content[row.start_idx:row.end_idx] != row.match_text:
                if strict:
                    raise ValueError(f'No match for {i} with {row.match_text} in article {article_id}')
                j = 0
                while j < 600 and (pd.isna(articles.loc[article_id+add+j].content) or articles.loc[article_id+add+j].content[row.start_idx:row.end_idx] != row.match_text):
                    j += 1
                add += j
                assert articles.loc[article_id+add].content[row.start_idx:row.end_idx] == row.match_text, (article_id, i, j)
                increases[article_id] = add
                pbar.set_description(f'{article_id} -> {add}')
                # print(article_id, add)
        for i, row in comp_occ[comp_occ.article_id == article_id].iterrows():
            assert articles.loc[article_id].content[row.start_idx:row.end_idx] == row.match_text
    return increases

# x = find_increases(people_occ_bb, articles)

In [ ]:
for i, row in tqdm(people_occ.iterrows(), total=len(people_occ)):
    assert articles.loc[row.article_id].content[row.start_idx:row.end_idx] == row.match_text

In [ ]:
comp_names = securities_ds.get_all_company_names()
assert all([com in comp_names.Name.values for com in people_occ.company.unique()]), 'Sanity Check: Valid company names'
assert all(people_occ.notna().any()), 'Sanity Check: No missing values'

TODO: Validate correctness article_ids, start_idx and end_idx

TODO: Repeat after Bloomberg was added

In [ ]:
for x in tqdm(comp_occ.match_text.values):
    assert not securities_ds.get_most_similar_company(x, debug=True)[1], x

Regarding Bloomberg Company Occ from Anacode:


- Reuters ID: 0 to 106518
- Bloomberg: b0 equals 106519, b448394 equals 554913 (shift by 106519)

__Problems:__
- Article matching: 2 should be 106520, 447759 should be 554904
- The difference of ids shift increases by 626 -> counter appears to increase ID again and again (same as for reuters)
- 490 companies (I only have 443)
- Many false positives like 169 (Watson Pharam matches to WLTW in article 108(+106519)

### Data

In [31]:
all_correlations = pd.read_csv('../data/preprocessed/econometrics/correlations.csv', index_col=[0, 1])
all_correlations.index.levels[1].name = None
all_correlations = all_correlations[[a < b for a, b in all_correlations.index]]
all_correlations.index = all_correlations.index.remove_unused_levels()

comp_occ = pd.read_csv('../data/preprocessed/occurrences/occurrences.csv', index_col=0).reset_index(drop=True)
# FIXME: Remove later
# comp_occ = comp_occ[comp_occ.article_id <= 106493]
comp_coocc = nlp_utils.get_cooccurrences(comp_occ).stack()
comp_coocc = comp_coocc[[a < b for a, b in comp_coocc.index]]
comp_coocc.index = comp_coocc.index.remove_unused_levels()

time: 43.4 s


In [6]:
# ', '.join(map(lambda x: f'{x[0]}: {x[1]}', x.items()))
increments_re = pd.Series({ 26944: 1, 55568: 2, 55624: 3, 56146: 4, 60323: 5, 68948: 6, 68978: 7, 72066: 8, 72779: 9, 74970: 10, 75163: 11, 75690: 12, 80466: 13, 81443: 14, 88575: 15, 89332: 16, 92712: 17, 94556: 18, 95271: 19, 95551: 20, 98957: 21, 99316: 22, 102077: 23, 102831: 24, 104671: 25 })
increments_bb = pd.Series({ 107371: 6, 107384: 12, 107401: 13, 107423: 16, 107437: 23, 107458: 27, 107487: 34, 107502: 36, 107522: 38, 107546: 43, 107563: 45, 107585: 50, 107628: 63, 107638: 69, 107644: 70, 107651: 76, 107657: 78, 107702: 94, 107707: 97, 107733: 106, 107779: 124, 107803: 130, 107845: 191, 107892: 271, 107930: 273, 107942: 275, 107955: 276, 107964: 278, 107978: 282, 107990: 286, 108046: 287, 108251: 288, 108528: 289, 108566: 290, 108633: 292, 108753: 293, 108874: 296, 109128: 297, 109261: 299, 109442: 300, 109536: 302, 109639: 306, 109708: 308, 109922: 309, 110074: 310, 110236: 311, 110446: 312, 110496: 313, 110624: 314, 110754: 315, 110898: 316, 111111: 318, 111293: 319, 111436: 321, 111494: 322, 111675: 324, 111731: 326, 111882: 327, 112465: 329, 112616: 330, 112647: 331, 113071: 332, 113176: 333, 113321: 335, 113766: 338, 113846: 339, 113850: 340, 113942: 341, 114207: 343, 114657: 346, 114932: 349, 114980: 351, 115126: 352, 115217: 353, 115336: 354, 115487: 356, 115704: 359, 115865: 360, 116048: 361, 116136: 362, 116510: 364, 116669: 369, 116681: 370, 116866: 372, 117270: 374, 117559: 375, 117687: 377, 118014: 379, 118808: 382, 119147: 384, 119236: 385, 119256: 386, 119688: 388, 119843: 389, 119969: 390, 120140: 392, 120292: 393, 120356: 394, 120712: 396, 120965: 399, 121265: 400, 121535: 401, 121684: 402, 121791: 404, 122103: 405, 122435: 407, 122654: 408, 122879: 409, 123177: 410, 123739: 411, 123770: 412, 124177: 415, 124317: 416, 124398: 417, 124570: 418, 124581: 419, 124669: 420, 124938: 421, 125002: 422, 125394: 423, 125508: 424, 125617: 426, 125771: 427, 125827: 428, 126015: 434, 126103: 435, 126588: 436, 126740: 437, 126879: 439, 127110: 442, 127322: 444, 127485: 445, 127825: 447, 127929: 448, 128181: 451, 128322: 454, 128379: 457, 128480: 458, 128798: 459, 129010: 462, 129084: 463, 129205: 464, 129391: 465, 129624: 469, 129875: 471, 129905: 472, 130211: 475, 130499: 477, 130623: 478, 130798: 479, 130833: 480, 131088: 485, 131110: 486, 131229: 487, 131515: 489, 148956: 490, 149498: 493, 149513: 494, 149517: 495, 149787: 508, 149856: 510, 150010: 511, 151114: 512, 151344: 513, 151462: 514, 151640: 515, 151964: 520, 152008: 521, 152159: 522, 152211: 523, 152417: 524, 163470: 525, 163653: 526, 163662: 528, 167219: 530, 167253: 533, 167292: 546, 167366: 554, 167408: 555, 293997: 566, 294038: 579, 294068: 587, 294114: 590, 416623: 592, 416691: 593, 416820: 595, 416865: 596, 474335: 597, 474439: 598, 474626: 599, 474676: 600, 474787: 603, 487231: 605, 487458: 607, 487571: 608, 487655: 609, 487659: 610, 487686: 611, 487723: 612, 487758: 614, 487784: 616, 487906: 622, 488022: 626 })

def get_increase(idx):
    increments = increments_re if idx < 106535 else increments_bb
    diffs = idx - increments.index
    if all(diffs < 0):
        return 0
    return increments.iloc[diffs[diffs >= 0].argmin()]

time: 233 ms


In [7]:
people_occ_re = pd.read_csv('../data/preprocessed/occurrences/occurrences-reuters-people-enriched.csv')
people_occ_re = people_occ_re[people_occ_re.columns[2:]]
people_occ_re.article_id += people_occ_re.article_id.apply(get_increase)

people_occ_bb = pd.read_csv('../data/preprocessed/occurrences/occurrences-bloomberg-people-enriched.csv', index_col=0)
people_occ_bb.article_id += 106519
people_occ_bb.article_id += people_occ_bb.article_id.apply(get_increase)

people_occ = pd.concat([people_occ_re, people_occ_bb])
people_occ.reset_index(drop=True, inplace=True)

name2symbol = dict([(com, securities_ds.get_company(com)) for com in people_occ.company.unique()])
people_occ['stock_symbol'] = people_occ.company.apply(name2symbol.get)
people_coocc = nlp_utils.get_cooccurrences(people_occ).stack()
people_coocc = people_coocc[[a < b for a, b in people_coocc.index]]

people_occ_train = people_occ[~people_occ.article_id.isin(article_ids_before_train)]
people_coocc_train = nlp_utils.get_cooccurrences(people_occ_train).stack()
people_coocc_train = people_coocc_train[[a < b for a, b in people_coocc_train.index]]

time: 5.8 s


In [ ]:
columns = ['article_id', 'start_idx', 'end_idx', 'stock_symbol']
mixed_occ = pd.concat([comp_occ[columns], people_occ[columns]]).sort_values(['article_id', 'start_idx']).reset_index(drop=True)
mixed_occ.drop_duplicates(['article_id', 'start_idx', 'end_idx'], inplace=True)
mixed_occ.reset_index(drop=True, inplace=True)

#### Only during train set

In [ ]:
TRAIN_FRAME = pd.to_datetime('2010-01-05'), pd.to_datetime('2012-05-21')
VAL_FRAME = pd.to_datetime('2012-05-22'), pd.to_datetime('2012-12-31')
NEWS = os.path.join("..", "data", "preprocessed", "news-v3.csv")
# articles = pd.read_csv(NEWS, index_col=0, nrows=people_occ.article_id.max()+50,)
articles_dates = pd.read_csv(NEWS, index_col=0, usecols=[0, 1, 2])
articles_dates.date = pd.to_datetime(articles_dates.date)
article_ids_before_train = set(articles_dates.index[articles_dates.date < TRAIN_FRAME[0]])

In [ ]:
# comp_occ_train = comp_occ[~comp_occ.article_id.isin(article_ids_before_train)].reset_index(drop=True)
# comp_coocc_train = nlp_utils.get_cooccurrences(comp_occ_train).stack()
# comp_coocc_train = comp_coocc_train[[a < b for a, b in comp_coocc_train.index]]

# people_occ_train = people_occ[~people_occ.article_id.isin(article_ids_before_train)]
# people_coocc_train = nlp_utils.get_cooccurrences(people_occ_train).stack()
# people_coocc_train = people_coocc_train[[a < b for a, b in people_coocc_train.index]]

- 40.702.147 Entities
- 9.636.490 tagged as ORG
- 463.595 linked to my stock companys (distributed over 126.583 articles)

Damerau-Levenshtein distance is not working (too repelling)

### Inspect Articles For Show Cases

In [ ]:
NEWS = os.path.join("..", "data", "preprocessed", "news-v3.csv")
# articles = pd.read_csv(NEWS, index_col=0, nrows=people_occ.article_id.max()+50,)
articles = pd.read_csv(NEWS, index_col=0)

# for i, row in tqdm(comp_occ.iterrows(), total=len(comp_occ)):
#     assert row.match_text == articles.loc[row.article_id].content[row.start_idx:row.end_idx], i

In [8]:
# _, occ_per_article = nlp_utils.get_cooccurrences(comp_occ, debug=True)

def merge_ranges(index_pairs):
    ranges = []
    for x, y in index_pairs:
        x = max(x, 0)
        if len(ranges) == 0:
            ranges.append([x, y])
        else:
            _, last_y = ranges[-1]
            if x <= last_y and y > last_y:
                ranges[-1][1] = y
            elif x > last_y: 
                ranges.append([x, y])
    return ranges

def get_matching_article(symA, symB):
    x = occ_per_article[[symA, symB]]
    return x[(x[symA] != 0) & (x[symB] != 0)].index.tolist()

def print_cooccurrences(symA, symB):
    x = occ_per_article[[symA, symB]]
    for idx, row in x[(x[symA] != 0) & (x[symB] != 0)].iterrows():
        print(f'--->Article: {idx}<----', row[symA], row[symB])
        temp_occs = comp_occ[(comp_occ.article_id == idx) & (comp_occ.stock_symbol.isin([symA, symB]))]
        # temp_occs.sort_values('start_idx', ascending=True, inplace=True)
        init_indeces = [(row.start_idx - 100, row.end_idx + 100) for i, row in temp_occs.iterrows()]
        indeces = merge_ranges(init_indeces)
        if len(init_indeces) == len(indices):
            continue
        for x, y in indeces:
            print('\n>>>', articles.loc[idx].content[x:y], '>>>\n')

time: 199 ms


In [ ]:
# match_articles = get_matching_article('CB', 'ALL')
match_articles = get_matching_article('IBM', 'MCD')
# 'ADP', 'XOM'
# print(articles.loc[match_articles[1]].content)
print_cooccurrences('IBM', 'MCD')

In [ ]:
merged[(merged['Same Industry'] == 0) & (merged.Cooccurrences > 5) & (merged.Residuals > 0.25)]

### Try New Distance Measure

In [18]:
len(comp_occ_train), len(comp_occ), len(people_occ_train), len(people_occ)

(367190, 463595, 16460, 19009)

time: 730 ms


In [ ]:
comp_distances = nlp_utils.get_distances(comp_occ)
comp_distances_train = nlp_utils.get_distances(comp_occ_train)
people_distances = nlp_utils.get_distances(people_occ)
# mixed_distances = nlp_utils.get_distances(mixed_occ)

In [34]:
comp_distances2 = nlp_utils.get_cheap_distances(comp_occ)
comp_distances_train2 = nlp_utils.get_distances(comp_occ_train)
people_distances2 = nlp_utils.get_cheap_distances(people_occ)
# mixed_distances2 = nlp_utils.get_cheap_distances(mixed_occ)

time: 19min 28s


### Correlate Features

In [115]:
def make_mergable(x):
    if len(x.shape) == 2:
        x = x.stack()
    return x[[a < b for a, b in x.index]]

merged = pd.concat([comp_coocc.rename('Cooccurrences'),  # 429
#                     comp_coocc_train.rename('Cooccurrences - Train'),  # 429
                    people_coocc.rename('Cooccurrences - People'),  # 200
#                     people_coocc_train.rename('Cooccurrences - People - Train'),  # 200
                    # mixed_coocc.rename('Cooccurrences - Mixed'),  # 200
                    make_mergable(comp_distances).rename('Dist'),
#                     make_mergable(comp_distances_train).rename('Dist - Train'),
                    make_mergable(people_distances).rename('Dist - People'),
                    # make_mergable(mixed_distances).rename('Dist - Mixed'),  # 450
                    make_mergable(comp_distances2).rename('Dist2'),
#                     make_mergable(comp_distances_train2).rename('Dist2 - Train'),
                    make_mergable(people_distances2).rename('Dist2 - People'),
                    # make_mergable(mixed_distances2).rename('Dist2 - Mixed'),
                    *[all_correlations[col].abs() for col in all_correlations.columns]  # 466
                   ], axis=1).dropna(subset=['Cooccurrences', 'Price']).fillna(0)
merged.index = merged.index.remove_unused_levels()
merged['Cooccurrences - Add'] = merged['Cooccurrences'] + merged['Cooccurrences - People']
merged['Dist - Add'] = merged['Dist'] + merged['Dist - People']
merged['Dist2 - Add'] = merged['Dist2'] + merged['Dist2 - People']

time: 4.29 s


In [192]:
print(len(all_correlations.index), len(all_correlations.index.levels[0]), len(all_correlations.index.levels[1]))
print(len(comp_coocc.index), len(comp_coocc.index.levels[0]), len(comp_coocc.index.levels[1]))
print(len(merged.index), len(merged.index.levels[0]), len(merged.index.levels[1]))

108811 466 466
97903 442 442
86736 416 416
time: 303 ms


In [86]:
merged.corr().round(4)

,Cooccurrences,Cooccurrences - People,Dist,Dist - People,Dist2,Dist2 - People,Price,Return,Normalized,Residuals,Same Industry,Cooccurrences - Add,Dist - Add,Dist2 - Add
Cooccurrences,1.0000,0.5239,0.1319,0.2279,0.1314,0.2183,0.0056,0.0588,0.0774,0.0799,0.0690,0.9999,0.1635,0.1594
Cooccurrences - People,0.5239,1.0000,0.0415,0.4456,0.0404,0.4648,-0.0009,0.0263,0.0363,0.0392,0.0393,0.5345,0.1081,0.1049
Dist,0.1319,0.0415,1.0000,0.0720,0.8595,0.0757,-0.0132,0.0930,0.1067,0.1079,0.1624,0.1315,0.9885,0.8524
Dist - People,0.2279,0.4456,0.0720,1.0000,0.0682,0.8835,-0.0011,0.0173,0.0312,0.0331,0.0362,0.2327,0.2219,0.1911
Dist2,0.1314,0.0404,0.8595,0.0682,1.0000,0.0767,-0.0356,0.0746,0.0916,0.0921,0.1447,0.1310,0.8506,0.9901
Dist2 - People,0.2183,0.4648,0.0757,0.8835,0.0767,1.0000,-0.0006,0.0201,0.0303,0.0319,0.0424,0.2234,0.2080,0.2157
Price,0.0056,-0.0009,-0.0132,-0.0011,-0.0356,-0.0006,1.0000,0.1783,0.0418,0.0398,0.0553,0.0055,-0.0131,-0.0350
Return,0.0588,0.0263,0.0930,0.0173,0.0746,0.0201,0.1783,1.0000,0.1296,0.1299,0.2679,0.0587,0.0936,0.0759
Normalized,0.0774,0.0363,0.1067,0.0312,0.0916,0.0303,0.0418,0.1296,1.0000,0.9882,0.1963,0.0773,0.1091,0.0940
Residuals,0.0799,0.0392,0.1079,0.0331,0.0921,0.0319,0.0398,0.1299,0.9882,1.0000,0.2013,0.0799,0.1105,0.0947


time: 319 ms


In [55]:
merged.corr(method='kendall').round(4)

,Cooccurrences,Cooccurrences - People,Dist,Dist - People,Dist2,Dist2 - People,Price,Return,Normalized,Residuals,Same Industry,Cooccurrences - Add,Dist - Add,Dist2 - Add
Cooccurrences,1.0000,0.0880,0.9383,0.0879,0.9141,0.0879,-0.0331,0.0485,0.0506,0.0505,0.1431,0.9957,0.9349,0.9118
Cooccurrences - People,0.0880,1.0000,0.0810,0.9990,0.0773,0.9985,0.0010,0.0154,0.0120,0.0127,0.0456,0.1380,0.1352,0.1383
Dist,0.9383,0.0810,1.0000,0.0810,0.9591,0.0810,-0.0312,0.0476,0.0497,0.0497,0.1426,0.9345,0.9949,0.9546
Dist - People,0.0879,0.9990,0.0810,1.0000,0.0773,0.9993,0.0010,0.0154,0.0120,0.0127,0.0456,0.1379,0.1353,0.1384
Dist2,0.9141,0.0773,0.9591,0.0773,1.0000,0.0773,-0.0319,0.0447,0.0478,0.0478,0.1366,0.9103,0.9546,0.9937
Dist2 - People,0.0879,0.9985,0.0810,0.9993,0.0773,1.0000,0.0010,0.0154,0.0120,0.0127,0.0456,0.1379,0.1352,0.1384
Price,-0.0331,0.0010,-0.0312,0.0010,-0.0319,0.0010,1.0000,0.1127,0.0186,0.0176,0.0456,-0.0329,-0.0309,-0.0316
Return,0.0485,0.0154,0.0476,0.0154,0.0447,0.0154,0.1127,1.0000,0.0570,0.0564,0.1876,0.0489,0.0480,0.0451
Normalized,0.0506,0.0120,0.0497,0.0120,0.0478,0.0120,0.0186,0.0570,1.0000,0.8735,0.1195,0.0504,0.0496,0.0478
Residuals,0.0505,0.0127,0.0497,0.0127,0.0478,0.0127,0.0176,0.0564,0.8735,1.0000,0.1213,0.0504,0.0497,0.0479


time: 2.08 s


#### News Correlations

In [44]:
columns = ['Residuals', 'Cooccurrences', 'Dist2 - Add', 'Dist - Add', 'Same Industry']
renamed_columns = ['Stock Correlation', 'Co-occurrences', 'Minimum Distance', 'Pairwise Distance', 'Same Industry']
final = merged[columns].rename(dict(zip(columns, renamed_columns)), axis=1)
final.corr().round(4)

,Stock Correlation,Co-occurrences,Minimum Distance,Pairwise Distance,Same Industry
Stock Correlation,1.0000,0.0799,0.0947,0.1105,0.2013
Co-occurrences,0.0799,1.0000,0.1594,0.1635,0.0690
Minimum Distance,0.0947,0.1594,1.0000,0.8622,0.1477
Pairwise Distance,0.1105,0.1635,0.8622,1.0000,0.1643
Same Industry,0.2013,0.0690,0.1477,0.1643,1.0000


time: 231 ms


In [45]:
final.corr(method='kendall').round(4)

,Stock Correlation,Co-occurrences,Minimum Distance,Pairwise Distance,Same Industry
Stock Correlation,1.0000,0.0505,0.0479,0.0497,0.1213
Co-occurrences,0.0505,1.0000,0.9118,0.9349,0.1431
Minimum Distance,0.0479,0.9118,1.0000,0.9591,0.1377
Pairwise Distance,0.0497,0.9349,0.9591,1.0000,0.1435
Same Industry,0.1213,0.1431,0.1377,0.1435,1.0000


time: 371 ms


#### Stock Correlations

In [46]:
math_utils.r_critical_values(len(merged))

0.1000    0.01
0.0500    0.01
0.0100    0.01
0.0010    0.01
0.0001    0.01
dtype: float64

time: 202 ms


In [47]:
merged[['Dist - Add', *all_correlations.columns]].rename({'Dist - Add': 'Pairwise Distance'}, axis=1).corr().round(4)

,Pairwise Distance,Price,Return,Normalized,Residuals,Same Industry
Pairwise Distance,1.0000,-0.0131,0.0936,0.1091,0.1105,0.1643
Price,-0.0131,1.0000,0.1783,0.0418,0.0398,0.0553
Return,0.0936,0.1783,1.0000,0.1296,0.1299,0.2679
Normalized,0.1091,0.0418,0.1296,1.0000,0.9882,0.1963
Residuals,0.1105,0.0398,0.1299,0.9882,1.0000,0.2013
Same Industry,0.1643,0.0553,0.2679,0.1963,0.2013,1.0000


time: 240 ms


In [48]:
merged[['Dist - Add', *all_correlations.columns]].rename({'Dist - Add': 'Pairwise Distance'}, axis=1).corr(method='kendall').round(4)

,Pairwise Distance,Price,Return,Normalized,Residuals,Same Industry
Pairwise Distance,1.0000,-0.0309,0.0480,0.0496,0.0497,0.1435
Price,-0.0309,1.0000,0.1127,0.0186,0.0176,0.0456
Return,0.0480,0.1127,1.0000,0.0570,0.0564,0.1876
Normalized,0.0496,0.0186,0.0570,1.0000,0.8735,0.1195
Residuals,0.0497,0.0176,0.0564,0.8735,1.0000,0.1213
Same Industry,0.1435,0.0456,0.1876,0.1195,0.1213,1.0000


time: 586 ms


#### Final

In [ ]:
print('\nPearson:\n')
print(utils.pandas_df_to_markdown_table(final.corr().round(4)))
print('\nSpearman:\n')
print(utils.pandas_df_to_markdown_table(final.corr(method='spearman').round(4)))
print('\nKendall:\n')
print(utils.pandas_df_to_markdown_table(final.corr(method='kendall').round(4)))

Pearson:

|        -          |   Stock Correlation |   Co-occurrences |   Minimum Distance |   Pairwise Distance |   Same Industry |
|:------------------|--------------------:|-----------------:|-------------------:|--------------------:|----------------:|
| Stock Correlation |              1      |           0.0923 |             0.1088 |              0.1156 |          0.2008 |
| Co-occurrences    |              0.0923 |           1      |             0.2271 |              0.2236 |          0.0821 |
| Minimum Distance  |              0.1088 |           0.2271 |             1      |              0.907  |          0.1675 |
| Pairwise Distance |              0.1156 |           0.2236 |             0.907  |              1      |          0.1668 |
| Same Industry     |              0.2008 |           0.0821 |             0.1675 |              0.1668 |          1      |

Spearman:

|        -          |   Stock Correlation |   Co-occurrences |   Minimum Distance |   Pairwise Distance |   Same Industry |
|:------------------|--------------------:|-----------------:|-------------------:|--------------------:|----------------:|
| Stock Correlation |              1      |           0.0681 |             0.0659 |              0.0666 |          0.1482 |
| Co-occurrences    |              0.0681 |           1      |             0.9894 |              0.99   |          0.1676 |
| Minimum Distance  |              0.0659 |           0.9894 |             1      |              0.9993 |          0.1663 |
| Pairwise Distance |              0.0666 |           0.99   |             0.9993 |              1      |          0.1677 |
| Same Industry     |              0.1482 |           0.1676 |             0.1663 |              0.1677 |          1      |

Kendall:

|        -          |   Stock Correlation |   Co-occurrences |   Minimum Distance |   Pairwise Distance |   Same Industry |
|:------------------|--------------------:|-----------------:|-------------------:|--------------------:|----------------:|
| Stock Correlation |              1      |           0.0546 |             0.0525 |              0.0532 |          0.121  |
| Co-occurrences    |              0.0546 |           1      |             0.9472 |              0.9549 |          0.1645 |
| Minimum Distance  |              0.0525 |           0.9472 |             1      |              0.9817 |          0.1621 |
| Pairwise Distance |              0.0532 |           0.9549 |             0.9817 |              1      |          0.1638 |
| Same Industry     |              0.121  |           0.1645 |             0.1621 |              0.1638 |          1      |

- Normalization partly removed correlation between stock return and industry
    - Return vs. Same Industry: 0.2717 --> Normalized vs. Same Industry: 0.1960
- Correlation between Stock and Cooc increased with each preprocessing step
- "Cooc" / "Add Cooc" correlates with "Residuals"
- Addition of People Dist and Comp Dist increased Correlation
- Spearman: Higher for intra-text features (Cooc, Distances, Same Industry) --> monotonic, non-linear relationship, Lower in comparison with stock correlation --> linear relationship between both feature types
- Kendall: All values are lower than spearman, same as spearman compared to pearson
- Pearson/Spearman Significance: 0.10 -> 0.0058, 0.05 -> 0.0069, 0.01 -> 0.0090
- Kendall Significance: 0.10 -> 0.0038, 0.05 -> 0.0046, 0.01 -> 0.0060

Plot/Print diagonal:
- https://seaborn.pydata.org/examples/many_pairwise_correlations.html